### The one pipeline
<img src="images/ring.www.playbuzz.com.jpg" width=60% style="display: inline-block">
<br>
image source: www.playbuzz.com

In [33]:
from classes.Data import Data
import pandas as pd 

class Pipeline(Data):
    
    def __init__(self,name, data_list,merge_type,store):
        self.data_list = data_list
        self.name = name
        self.merge_type = merge_type
        self.store = store
        self.df = pd.DataFrame()
        
    def extract(self):
        list(map(lambda x:x.extract(),self.data_list))
        merge = self.merge_type(self.name,self.data_list,self.store)
        
        merge.extract()
        merge.put()
        self.df = merge.df
    
    def put(self):
        if (self.df.empty) :
            print("Pipeline.put(): dataframe is empty, run get() or extract()")
            return
        self.storage.put(self.df, self.name)
    
    def get(self):
        self.df = self.storage.get(self.name)
        
        
        

In [7]:
from classes.DistanceMergeData import DistanceMergeData
from classes.ScraperData import ScraperData
from classes.RidbData import RidbData
from classes.RidbDataLive import RidbDataLive

# import the scrapers
from classes.ReservationsScraper import ReservationsScraper
from classes.UsfsWebScraper import UsfsWebScraper

# import the storage
from classes.MysqlStorage import MysqlStorage
from classes.CsvStorage import CsvStorage
from classes.Pipeline import Pipeline

# import visualization
from classes import BokehPlot
from bokeh.io import output_notebook
from bokeh.plotting import show

import config
import pandas as pd

In [8]:
mysql_store = MysqlStorage()
csv_store = CsvStorage()
destination_info = dict(Latitude=45.4977712, Longitude=-121.8211673, radius=15)
start_date = '06/01/2016'
stay_length = 2
usfs_urls = pd.read_csv('data/usfs_sites.csv')
reservation_urls = pd.read_csv('data/reservation_urls.csv')
reservation_urls = reservation_urls.assign(start_date=start_date, stay_length=stay_length)

In [9]:
ridb_data = RidbData('ridb_merge_lab', "camping", destination_info, mysql_store)
ridb_data_live = RidbDataLive('ridb_live_merge_lab', "camping", destination_info, mysql_store)
usfs_data = ScraperData('usfs_merge_lab',usfs_urls,mysql_store,UsfsWebScraper())
res_data = ScraperData('res_merge_lab',reservation_urls,mysql_store,ReservationsScraper())

starting display


In [10]:
one_data_list = [res_data, usfs_data,ridb_data]
# live version
# one_data_list = [res_data, usfs_data,ridb_hiking_live, ridb_data_live]

In [12]:
pipe = Pipeline('the_one_pipelne', one_data_list, DistanceMergeData,mysql_store)

In [13]:
pipe.extract()

getting reservation.html
setting up web browser
browsed to reservation.html
browsed to reservation.html
browsed to reservation.html
browsed to reservation.html
browsed to reservation.html
browsed to reservation.html
closing browser
drop least recent
5    LOST  LAKE  RESORT  AND  CAMPGROUND
Name: FacilityName, dtype: object
drop first
Badger  Lake  Campground


In [14]:
pipe.df.columns

Index(['FacilityAdaAccess', 'FacilityDescription', 'FacilityDirections',
       'FacilityEmail', 'FacilityID', 'FacilityLatitude', 'FacilityLongitude',
       'FacilityMapURL', 'FacilityName', 'FacilityPhone',
       'FacilityReservationURL', 'FacilityTypeDescription',
       'FacilityUseFeeDescription', 'Keywords', 'LastUpdatedDate',
       'LegacyFacilityID', 'OrgFacilityID', 'StayLimit', 'merge_index',
       'CurrentConditions', 'OpenSeason', 'Reservations', 'Restroom', 'Water',
       'FacilityElevation', 'FacilityStatus', 'SitesAvailable'],
      dtype='object')

In [15]:
output_notebook()

Loading BokehJS ...

In [16]:
cols_to_display=['FacilityName','SitesAvailable', 'Water','Restroom']

In [17]:
sites_with_availability = pipe.df.dropna(subset=['SitesAvailable'])

In [28]:
res_data.df

,FacilityLatitude,FacilityLongitude,FacilityName,SitesAvailable
0,44.688294,-122.010343,whispering falls campground,9
1,45.500800,-121.816410,lost lake campground,7
2,45.073611,-121.956944,lake harriet campground,6
3,45.179450,-121.696750,clear lake campground,3
4,45.305236,-121.867275,camp creek campground,9
5,45.642244,-121.925284,eagle creek campground,8


In [27]:
pipe.df[['FacilityName', 'Reservations', 'Restroom','SitesAvailable']]

,FacilityName,Reservations,Restroom,SitesAvailable
0,WILDWOOD RECREATION SITE,NaN,NaN,NaN
1,Wildwood Campground,NaN,NaN,NaN
2,WHISPERING FALLS CAMPGROUND,NaN,NaN,NaN
3,Lost Lake Resort,This site can be reserved by calling Toll Free...,Vault Toilet (18),7
4,Lost Lake Campground,NaN,NaN,NaN
6,LAKE HARRIET,This site can be reserved by calling Toll Free...,Vault Toilet,6
7,Kinnikinnick (Laurance Lake) Campground,None,Vault Toilet (2),NaN
8,Fifteenmile Campground,No reservations,Vault Toilet (1),NaN
9,CLEAR LAKE (OR),NaN,NaN,NaN
10,CLEAR LAKE CABIN LOOKOUT,NaN,NaN,NaN


In [18]:
p = BokehPlot.create_plot(sites_with_availability, cols_to_display)

FacilityName
SitesAvailable
Water
Restroom


In [19]:
show(p)